#  Create Agents to Research and Write an Article¶


In [1]:
# Install libraries
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

  Using cached typer-0.9.4-py3-none-any.whl.metadata (14 kB)


  Using cached huggingface_hub-0.16.4-py3-none-any.whl.metadata (12 kB)
  Using cached websockets-12.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)


Using cached typer-0.9.4-py3-none-any.whl (45 kB)
Using cached huggingface_hub-0.16.4-py3-none-any.whl (268 kB)
Using cached websockets-12.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (130 kB)
  Attempting uninstall: websockets
    Found existing installation: websockets 11.0.3
    Uninstalling websockets-11.0.3:
      Successfully uninstalled websockets-11.0.3
  Attempting uninstall: typer
    Found existing installation: typer 0.12.3
    Uninstalling typer-0.12.3:
      Successfully uninstalled typer-0.12.3
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.23.3
    Uninstalling huggingface-hub-0.23.3:
      Successfully uninstalled huggingface-hub-0.23.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 4.36.1 requires huggingface-hub>=0.19.3, but you have hug

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Import from crewAI library
from crewai import Agent, Task, Crew

In [6]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [7]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [8]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [9]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [10]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [11]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [12]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [13]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew
**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [18]:
result = crew.kickoff(inputs={"topic": "Multi AI agent system"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Multi AI agent system.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Content Plan: Multi AI Agent System

Introduction:
- Definition of Multi AI Agent System
- Importance and applications in various industries
- Brief overview of key players and recent trends

Key Points:
1. Latest Trends in Multi AI Agent System:
- Advancements in multi-agent learning algorithms
- Integration of multiple AI agents for complex decision-making processes
- Collaborative learning among AI agents for improved efficiency

2. Key Players in Multi AI Agent System:
- Google with its DeepMind project
- O

I now can give a great answer

Final Answer:

# Embracing the Future: The Rise of Multi AI Agent Systems

**Introduction:**

In the dynamic landscape of artificial intelligence, Multi AI Agent System stands out as a powerful approach that leverages the collaboration of multiple AI agents to tackle complex decision-making processes. This innovative system has gained significant importance and found diverse applications across various industries, offering enhanced efficiency and performance. Key players such as Google's DeepMind project, OpenAI, and IBM Watson are at the forefront of developing cutting-edge AI agent solutions, driving the latest trends in this evolving field.

**Latest Trends in Multi AI Agent System:**

Advancements in multi-agent learning algorithms have revolutionized the capabilities of AI agents, enabling them to learn collaboratively and make more informed decisions. The integration of multiple AI agents for complex tasks has opened up new possibilities for process

- Display the results of your execution as markdown in the notebook.

In [19]:
from IPython.display import Markdown
Markdown(result)

# Embracing the Future: The Rise of Multi AI Agent Systems

**Introduction:**

In the dynamic landscape of artificial intelligence, Multi AI Agent System stands out as a powerful approach that leverages the collaboration of multiple AI agents to tackle complex decision-making processes. This innovative system has gained significant importance and found diverse applications across various industries, offering enhanced efficiency and performance. Key players such as Google's DeepMind project, OpenAI, and IBM Watson are at the forefront of developing cutting-edge AI agent solutions, driving the latest trends in this evolving field.

**Latest Trends in Multi AI Agent System:**

Advancements in multi-agent learning algorithms have revolutionized the capabilities of AI agents, enabling them to learn collaboratively and make more informed decisions. The integration of multiple AI agents for complex tasks has opened up new possibilities for process optimization and problem-solving. Collaborative learning among AI agents further enhances their efficiency and adaptability, making them invaluable assets in the realm of artificial intelligence.

**Key Players in Multi AI Agent System:**

Google's DeepMind project has been a trailblazer in pushing the boundaries of AI research and development, particularly in the realm of multi-agent systems. OpenAI's diverse portfolio of AI agents for various applications showcases the versatility and potential of this technology. IBM Watson's AI agent solutions for businesses have demonstrated significant impact in optimizing processes and driving innovation across industries.

**Noteworthy News and Case Studies:**

Recent research studies have highlighted the potential of multi-agent systems in revolutionizing healthcare processes and improving patient outcomes. Industry collaborations are actively exploring the implementation of AI agents in supply chain management to enhance efficiency and streamline operations. Government initiatives to regulate the use of AI agents in sensitive sectors reflect the growing importance of ethical considerations in deploying advanced technologies.

**Benefits and Challenges of Multi AI Agent System:**

The benefits of Multi AI Agent System are vast, ranging from improved decision-making to enhanced efficiency and productivity. However, challenges such as the complexity of integrating multiple AI agents and the need for practical use cases pose significant obstacles to widespread adoption. Addressing these challenges will be crucial in unlocking the full potential of multi-agent systems and driving innovation in the field of artificial intelligence.

**Future Outlook and Predictions:**

Looking ahead, the future of Multi AI Agent System holds immense promise for revolutionizing industries and transforming the way we approach complex problems. As advancements in AI technology continue to accelerate, we can expect to see even greater collaboration among AI agents, leading to unprecedented levels of efficiency and innovation. The integration of AI agents into various facets of our daily lives will reshape the way we work, communicate, and interact with technology.

**Call to Action:**

For further reading and implementation of Multi AI Agent System, I recommend exploring resources such as "Multi-Agent Systems: A Modern Approach to Distributed Artificial Intelligence" by Gerhard Weiss and industry reports on AI adoption and trends from leading research firms. Stay informed, stay curious, and embrace the future of artificial intelligence with Multi AI Agent System.

In conclusion, the rise of Multi AI Agent System represents a paradigm shift in the field of artificial intelligence, offering limitless possibilities for innovation and advancement. By staying informed and actively engaging with the latest trends and developments in this exciting field, we can unlock the full potential of AI agents and drive meaningful change across industries. Embrace the future, embrace Multi AI Agent System.